### 0. Imports and Generators

In [1]:
import math
import numpy as np
rng = np.random.default_rng()

### 1. Function for the volume of the unit sphere and approximation of $\pi$

In [2]:
def volume_unit_sphere(dim: int, N: int = 100000) -> float:
    sample = rng.uniform(-1, 1, (N, dim))
    M = np.sum(np.linalg.norm(sample, axis=1) <= 1)
    return 2**dim * M / N

The volume of the 2-dimensional unit sphere is exactly $\pi$. Therefore, $\pi$ can be approximated directly by calling `volume_unit_sphere(2)`.

In [3]:
volume_unit_sphere(2)

3.14576

### 2. Accuracy for different sample sizes

We want to estimate the minimum $N$ for which $\pi$ is approximated within a certain accuracy $\varepsilon$. To find this 
$N$, we perform a binary search over powers of 10 to identify the smallest one that meets the desired accuracy. As the Monte‑Carlo simulation is inherently random, we run multiple independent trials and require the approximation to lie within $\varepsilon$ in at least the fraction specified by the confidence level. This ensures the chosen $N$ attains the desired accuracy with high probability.

In [4]:
def min_sample_size(epsilon: float, dim: int = 2, confidence: float = 0.98, max_trials: int = 50) -> int:    
    gamma = math.gamma(dim / 2 + 1)
    max_fails = int((1 - confidence) * max_trials)

    def check_accuracy(N: int) -> bool:
        fails = 0
        for _ in range(max_trials):
            V = volume_unit_sphere(dim, N=N)
            pi_approx = (V * gamma)**(2 / dim)
            if np.abs(pi_approx - np.pi) >= epsilon:
                fails += 1
                if fails > max_fails:
                    return False
        return True
    
    exp_low, exp_high = 3, 9
    while exp_high - exp_low > 1:
        exp = (exp_high + exp_low) // 2
        if check_accuracy(10**exp):
            exp_high = exp
        else:
            exp_low = exp

    return 10**exp_high

Now we can determine the approximate sample sizes $N_1$ and $N_2$ required to achieve an accuracy of less than $\varepsilon_1 = 0.01$ and $\varepsilon_2 = 0.001$. (Note: The calculation of $N_2$ may take up to 4 minutes.)

In [5]:
N_1 = min_sample_size(0.01)
N_2 = min_sample_size(0.001)
N_1, N_2

(1000000, 100000000)

We obtain $N_1 \simeq 10^6$ and $N_2 \simeq 10^8$. We can deduce that $\varepsilon \propto \frac{1}{\sqrt{N}}$.

In [6]:
# N_1, N_2 = 10**6, 10**8
np.pi, volume_unit_sphere(2, N=N_1), volume_unit_sphere(2, N=N_2)

(3.141592653589793, 3.142204, 3.14153452)

### 3. Accuracy for higher dimensions

For higher dimensions (e.g. $n = 10$), the ratio of the sphere's volume to the enclosing cube's volume decreases exponentially with dimension. For $n = 10$ this ratio already is $<0.03$. Because of the limited sample size it therefore becomes increasingly unlikely to generate points inside the sphere. As a result the approximation becomes significantly less accurate.

In [7]:
V = volume_unit_sphere(10)
pi_approx = (V * math.gamma(10 / 2 + 1))**(2 / 10)
V, pi_approx

(2.77504, 3.195141519360822)